## AVANCE 3  
### PI 1) Leer los csv

In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

categories = pd.read_csv('../Data/categories.csv')
cities = pd.read_csv('../Data/cities.csv')
countries = pd.read_csv('../Data/countries.csv')
customers = pd.read_csv('../Data/customers.csv')
employees = pd.read_csv('../Data/employees.csv', parse_dates=['HireDate', 'BirthDate'])
products = pd.read_csv('../Data/products.csv')
sales = pd.read_csv('../Data/sales.csv', parse_dates=['SalesDate'])


### PI 2) Utilizar la fórmula TotalPriceCalculated=(Quantity×UnitPrice)×(1−Discount)
y detectar los outliers en la columna ventas totales utilizando el rango intercuartílico

Primero hago la unión con productos para obtener el precio por producto en la variable df, valido que no haya duplicados en products para que no me duplique las filas en df.

In [2]:
df = sales.merge(products[['ProductID', 'Price']], on='ProductID', how='left', validate='m:1')
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98


Calculo los totales de precio por cantidad con el descuento

In [3]:
df['CalculatedPrice'] = (df['Quantity'] * df['Price']) * (1 - df['Discount'])
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,CalculatedPrice
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44"
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01"
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80


Calculo los Outliers

In [4]:
# Calculo IQR
q1 = df['CalculatedPrice'].quantile(0.25)
q3 = df['CalculatedPrice'].quantile(0.75)
iqr = q3 - q1

# Calculo los límites inferior y superior
lowerLimit = q1 - 1.5 * iqr
upperLimit = q3 + 1.5 * iqr

# Identifico si está fuera de los límites y calculo cuántos Outliers hay
df['IsOutlier'] = ((df['CalculatedPrice'] < lowerLimit) | (df['CalculatedPrice'] > upperLimit))
print ("Cantidad de Outliers: ", df['IsOutlier'].sum())
df.head()
df['IsOutlier'].value_counts()


Cantidad de Outliers:  48217


IsOutlier
False    6709908
True       48217
Name: count, dtype: int64

## Respuesta:  
### Se encontraron 48.217 Outliers en el precio total

### Verificar en qué horas se vende más

In [5]:
# 3) Extraer HORA desde SalesDate y encontrar la hora con más ventas totales
# Crear columna Hour (0–23)
df['Hour'] = df['SalesDate'].dt.hour

# Ventas totales por hora (suma de TotalPriceCalculated)
salesByHour = df.groupby('Hour', dropna=False)['CalculatedPrice'].sum().sort_values(ascending=False)
print(salesByHour.head(5))


Hour
16.00   179,014,421.24
20.00   178,949,163.88
2.00    178,420,846.95
6.00    178,381,199.01
19.00   178,346,115.33
Name: CalculatedPrice, dtype: float64


## Respuesta:  
### Las horas en que se registraron más ventas fueron las 16.00, 20.00, 2.00, 6.00 y 19.00 horas

### PI 3) ¿La empresa vende más durante los días de semana o en el fin de semana? Clasifica cada venta como Entre semana o Fin de semana a partir de SalesDate y compara el total de ventas entre ambos grupos. ¿En cuál se vende más?

In [6]:
# Primero calculo el día de la semana
df['DayOfWeek'] = df['SalesDate'].dt.day_of_week

# Calculo si es día de semana o fin de semana
df['TypeOfDate'] = df['DayOfWeek'].apply(lambda x: 'weekday' if x < 5 else 'weekend')

# Agrupo por TypeOfDate y sumo las ventas
ventasPorTipoDeDia = df.groupby('TypeOfDate')['CalculatedPrice'].sum().reset_index()
print(ventasPorTipoDeDia)

  TypeOfDate  CalculatedPrice
0    weekday 3,080,351,837.91
1    weekend 1,235,915,841.01


## Respuesta:  
### Se vende casi el triple los días de semana que durante el fin de semana.

## PI 4) Trabajo en DataFrame  
### Como parte del proceso de feature engineering, en el mismo DataFrame en el que vienes trabajando, calcula dos nuevas columnas en el dataset de ventas: (1) La edad del empleado al momento de su contratación y (2) años de experiencia al momento de realizar cada venta. Utiliza las columnas BirthDate, HireDate (de la tabla employees) y SalesDate (de la tabla sales). Asegúrate de trabajar con fechas en formato adecuado.

In [7]:
# Realizo la unión con Employers. Las fechas las validé al cargar los csv.

df = df.merge(employees[['EmployeeID', 'HireDate', 'Gender', 'BirthDate']], 
                 left_on='SalesPersonID', right_on='EmployeeID',
                 how='left', validate='m:1')
df = df.rename(columns={'Gender': 'EmployeeGender', 'HireDate': 'EmployeeHireDate', 'BirthDate': 'EmployeeBirthDate'})
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,CalculatedPrice,IsOutlier,Hour,DayOfWeek,TypeOfDate,EmployeeID,EmployeeHireDate,EmployeeGender,EmployeeBirthDate
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64,False,7.00,0.00,weekday,6,2013-06-22 13:20:18.080,M,1987-01-13
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82,False,16.00,4.00,weekday,16,2017-02-10 11:21:26.650,M,1951-07-07
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44",False,19.00,3.00,weekday,13,2011-12-12 10:43:52.940,M,1963-04-18
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01",False,14.00,5.00,weekend,8,2014-10-14 23:12:53.420,M,1956-12-13
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80,False,15.00,0.00,weekday,10,2012-07-23 15:02:12.640,M,1963-12-30


### Calculo la edad del empleado en el momento de contratación

In [8]:
years = df['EmployeeHireDate'].dt.year - df['EmployeeBirthDate'].dt.year
before_birthday = (
    (df['EmployeeHireDate'].dt.month < df['EmployeeBirthDate'].dt.month) |
    ((df['EmployeeHireDate'].dt.month == df['EmployeeBirthDate'].dt.month) &
     (df['EmployeeHireDate'].dt.day < df['EmployeeBirthDate'].dt.day))
).astype(int)

df['EmployeeAgeAtHire'] = (years - before_birthday).astype('Int64')
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,CalculatedPrice,IsOutlier,Hour,DayOfWeek,TypeOfDate,EmployeeID,EmployeeHireDate,EmployeeGender,EmployeeBirthDate,EmployeeAgeAtHire
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64,False,7.00,0.00,weekday,6,2013-06-22 13:20:18.080,M,1987-01-13,26
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82,False,16.00,4.00,weekday,16,2017-02-10 11:21:26.650,M,1951-07-07,65
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44",False,19.00,3.00,weekday,13,2011-12-12 10:43:52.940,M,1963-04-18,48
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01",False,14.00,5.00,weekend,8,2014-10-14 23:12:53.420,M,1956-12-13,57
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80,False,15.00,0.00,weekday,10,2012-07-23 15:02:12.640,M,1963-12-30,48


### Calculo la antigüedad al momento de la venta


In [9]:
years = df['SalesDate'].dt.year - df['EmployeeHireDate'].dt.year
before_experience = (
    (df['SalesDate'].dt.month < df['EmployeeHireDate'].dt.month) |
    ((df['SalesDate'].dt.month == df['EmployeeHireDate'].dt.month) &
     (df['SalesDate'].dt.day < df['EmployeeHireDate'].dt.day))
).astype(int)

df['EmployeeExperience'] = (years - before_experience).astype('Int64')
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,IsOutlier,Hour,DayOfWeek,TypeOfDate,EmployeeID,EmployeeHireDate,EmployeeGender,EmployeeBirthDate,EmployeeAgeAtHire,EmployeeExperience
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,False,7.00,0.00,weekday,6,2013-06-22 13:20:18.080,M,1987-01-13,26,4
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,False,16.00,4.00,weekday,16,2017-02-10 11:21:26.650,M,1951-07-07,65,0
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,False,19.00,3.00,weekday,13,2011-12-12 10:43:52.940,M,1963-04-18,48,6
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,False,14.00,5.00,weekend,8,2014-10-14 23:12:53.420,M,1956-12-13,57,3
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,False,15.00,0.00,weekday,10,2012-07-23 15:02:12.640,M,1963-12-30,48,5


## PI 5 - Dataset
### Prepara un único dataset definitivo para modelado que combine información relevante de las tablas disponibles. Incluye las features que se han calculado previamente. Aplica transformaciones adecuadas a las variables categóricas y a las variables numéricas (si lo consideras necesario) para dejar los datos listos para ser utilizados por un modelo de machine learning. Justifica las transformaciones realizadas.La(opens in a new tab) variable objetivo es TotalPriceCalculated, por lo que debe quedar sin transformaciones.


In [10]:
# Agrego algunas columnas de productos y categorías

df = df.merge(products [['ProductID', 'CategoryID', 'Class', 'Resistant', 'IsAllergic']],
              on='ProductID',
              how='left')

df = df.merge(categories[['CategoryID', 'CategoryName']], 
              on='CategoryID',
              how='left')


In [11]:
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,EmployeeHireDate,EmployeeGender,EmployeeBirthDate,EmployeeAgeAtHire,EmployeeExperience,CategoryID,Class,Resistant,IsAllergic,CategoryName
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,2013-06-22 13:20:18.080,M,1987-01-13,26,4,1.00,High,Unknown,Unknown,Confections
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,2017-02-10 11:21:26.650,M,1951-07-07,65,0,8.00,Low,Durable,FALSE,Grain
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,2011-12-12 10:43:52.940,M,1963-04-18,48,6,11.00,Medium,Durable,TRUE,Produce
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,2014-10-14 23:12:53.420,M,1956-12-13,57,3,6.00,High,Durable,TRUE,Seafood
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,2012-07-23 15:02:12.640,M,1963-12-30,48,5,9.00,Low,Weak,FALSE,Poultry


In [12]:
# Agrego Label encoding para sexo del vendedor y tipo de dia de semana de venta
from sklearn.preprocessing import LabelEncoder

labelColumns = {'EmployeeGender', 'TypeOfDate'}
labelEncoders = {}

for col in labelColumns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    labelEncoders[col] = le

df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,EmployeeHireDate,EmployeeGender,EmployeeBirthDate,EmployeeAgeAtHire,EmployeeExperience,CategoryID,Class,Resistant,IsAllergic,CategoryName
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,2013-06-22 13:20:18.080,1,1987-01-13,26,4,1.00,High,Unknown,Unknown,Confections
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,2017-02-10 11:21:26.650,1,1951-07-07,65,0,8.00,Low,Durable,FALSE,Grain
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,2011-12-12 10:43:52.940,1,1963-04-18,48,6,11.00,Medium,Durable,TRUE,Produce
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,2014-10-14 23:12:53.420,1,1956-12-13,57,3,6.00,High,Durable,TRUE,Seafood
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,2012-07-23 15:02:12.640,1,1963-12-30,48,5,9.00,Low,Weak,FALSE,Poultry
